# Supply Chain Analytics - Two-product Transportation Problem

In this problem, there is a set $L$ of six locations labelled A through F. Each location is connected to each other one by a shipping link $(i,j)$ forming a network $N$, and the cost of shipping ($C_{ij}$) one ton of chemical from a location $i$ to another location $j$ is known. For example, it costs 7 dollars to send one ton of chemical from location A to location B, but only 4 dollars to ship a ton from location B to location A.  There are two chemicals, Chemical 1 and Chemical 2, but the shipping cost is the same for each. However, each link has a limited capacity $K_{ij}$.  For example, no more than 9 tons may be shipped from A to B, while no more than 16 tons may be shipped from B to A.

Each location has its own supply of each chemical ($S1_{i},S2_{i}$) and its own demand for each chemical ($D1_{i},D2_{i}$). We need to find the shipping schedule that minimizes overall shipping cost. Let $x_{ij}$ and $y_{ij}$ represent the tons of chemical 1 and chemical 2, respectively, shipped from location $i$ to location $j$.

\begin{equation*}
\begin{aligned}
& {\text{minimize}}
& & \sum_{(i,j) \in N}(x_{ij}+y_{ij})C_{ij} & \color{blue}{\longrightarrow \textbf{Shipping cost minimization (1)}}\\[2mm]
& \text{subject to}&&\\[2mm]
& & & x_{ij}+y_{ij}  \leq  K_{ij}, \text{ for } (i,j) \in N & \color{blue}{\longrightarrow \textbf{Arcs capacity (2)}}\\[2mm]
& & & S1_{i}+\sum_{j \in L | i\neq j}x_{ji}-\sum_{j \in L | i\neq j}x_{ij} \geq D1_{i}, \text{ for } i \in L & \color{blue}{\longrightarrow \textbf{Chem1 demand satisfaction (3)}}\\[2mm]
& & & S2_{i}+\sum_{j \in L | i\neq j}y_{ji}-\sum_{j \in L | i\neq j}y_{ij} \geq D2_{i}, \text{ for } i \in L & \color{blue}{\longrightarrow \textbf{Chem2 demand satisfaction (4)}}\\[2mm]
& & & x_{ij},y_{ij}  \in \mathbb{Z}_{+}, \text{ for } (i,j) \in N & \color{blue}{\longrightarrow \textbf{Nonegative integer variables (5)}}\\[2mm]
\end{aligned}
\end{equation*}

In [1]:
# Import PuLP modeller functions
from pulp import *

In [2]:
# Define list of locations
L = ["A","B","C","D","E","F"]

# Define supplies and demands
S1 = {"A":28,"B":0, "C":22,"D":2,"E":30,"F":0}
D1 = {"A":16,"B":11,"C":0, "D":0,"E":0, "F":6}
S2 = {"A":0, "B":30,"C":15,"D":0,"E":0, "F":0}
D2 = {"A":0, "B":18,"C":12,"D":0,"E":12,"F":0}

Arcs_Def = [(i,j) for i in L for j in L if i!=j]

C = {('A', 'B'):7, ('A', 'C'):9,  ('A', 'D'):6,  ('A', 'E'):5, ('A', 'F'):9,
     ('B', 'A'):4, ('B', 'C'):10, ('B', 'D'):1,  ('B', 'E'):1, ('B', 'F'):4,
     ('C', 'A'):5, ('C', 'B'):8,  ('C', 'D'):1,  ('C', 'E'):1, ('C', 'F'):1,
     ('D', 'A'):6, ('D', 'B'):6,  ('D', 'C'):1,  ('D', 'E'):6, ('D', 'F'):5,
     ('E', 'A'):3, ('E', 'B'):7,  ('E', 'C'):10, ('E', 'D'):6, ('E', 'F'):8,
     ('F', 'A'):8, ('F', 'B'):3,  ('F', 'C'):5,  ('F', 'D'):3, ('F', 'E'):9}

K = {('A', 'B'):9,  ('A', 'C'):17, ('A', 'D'):16, ('A', 'E'):9,  ('A', 'F'):13,
     ('B', 'A'):16, ('B', 'C'):20, ('B', 'D'):16, ('B', 'E'):10, ('B', 'F'):5,
     ('C', 'A'):13, ('C', 'B'):5,  ('C', 'D'):11, ('C', 'E'):12, ('C', 'F'):19,
     ('D', 'A'):19, ('D', 'B'):10, ('D', 'C'):8,  ('D', 'E'):1,  ('D', 'F'):6,
     ('E', 'A'):2,  ('E', 'B'):4,  ('E', 'C'):14, ('E', 'D'):14, ('E', 'F'):6,
     ('F', 'A'):9,  ('F', 'B'):14, ('F', 'C'):16, ('F', 'D'):7,  ('F', 'E'):17}

In [3]:
#Problem definition
prob = LpProblem("Two-product-Transportation",LpMinimize)

In [4]:
# Decision Variables (integer variables)
x = LpVariable.dicts("Chem1",Arcs_Def,0,None,LpInteger)
y = LpVariable.dicts("Chem2",Arcs_Def,0,None,LpInteger)

In [5]:
#Objective Function
prob += lpSum([(x[a] + y[a])*C[a] for a in Arcs_Def]), "Sum of Shipment Costs"

In [6]:
#Constraints (2)
for a in Arcs_Def:
    prob += x[a] + y[a] <= K[a], "Capacity arc" + str(a)   

In [7]:
#Constraints (3) & (4)
for l in L:
    prob += S1[l] + lpSum([x[a] for a in Arcs_Def if a[1] == l]) - lpSum([x[a] for a in Arcs_Def if a[0] == l]) >= D1[l], "Net in Chem1  %s"%l
    prob += S2[l] + lpSum([y[a] for a in Arcs_Def if a[1] == l]) - lpSum([y[a] for a in Arcs_Def if a[0] == l]) >= D2[l], "Net in Chem2  %s"%l

In [8]:
#Solve IP
prob.writeLP("Two-ProductTransportation.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [9]:
#Print optimal objective function value
print("Optimal Cost=", "$"+ str(value(prob.objective)))

Optimal Cost= $62.0


In [10]:
#Print optimal solution
for variable in prob.variables():
    if variable.varValue > 0:
        print ("{}* = {}".format(variable.name, variable.varValue))

Chem1_('C',_'F')* = 17.0
Chem1_('F',_'B')* = 11.0
Chem2_('B',_'E')* = 10.0
Chem2_('C',_'E')* = 2.0
